In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [52]:
print_Debt = True

In [2]:
quantum_columns = str(export_from_RISKCUSTOM("""select *from "RISKACCESS"."quantumDebt" fetch first 1 rows only""").columns.tolist()).replace('\'','\"')[1:-1]
dates = ['31/07/24', '31/08/24']
dates_SQL = str([f'TO_DATE(\'{item}\', \'DD/MM/YY\')' for item in dates])[1:-1].replace('\"','')
dates_SQL

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


"TO_DATE('31/07/24', 'DD/MM/YY'), TO_DATE('31/08/24', 'DD/MM/YY')"

In [3]:
# quantum debt
query = f"""
with table_1 as
(select 
{quantum_columns},
--"reportDate","effectiveFrom","entity","dealNo","holding","counterparty","amount","eventCurrency","amountUSD","actionDate","dealClass1","dealClass2","instrumentOwner","dealSet","facility", 
MAX("effectiveFrom") OVER (partition BY "reportDate") as "max_effectiveFrom"
from "RISKACCESS"."quantumDebt" 
where "reportDate" in ({dates_SQL})
--fetch first 10 rows only
)
select * from table_1
where "effectiveFrom" = table_1."max_effectiveFrom"
--fetch first 10 rows only
"""
data_Debt_export = export_from_RISKCUSTOM(query)
data_Debt_export.head(2)

,##batch_id,##deleted,##inserted,##origin_name,flag,actionDate,amount,eventCurrency,amountUSD,currentRate,...,debtRelated,reportDate,eventNumber,entitySAPBUCode,counterpartySAPID,counterpartySWIFTBICCode,effectiveFrom,xlsDate,loadEventId,max_effectiveFrom
0,40206330@WHPROD,N,2024-09-16 23:15:31,QUANTUM,-,2044-04-25,-3177325.24,RUB,-36804.416078,3.0,...,Y,2024-07-31,200548,V100,1000147688,None,2024-09-16,2024-09-16 23:14:48,209390,2024-09-16
1,40206330@WHPROD,N,2024-09-16 23:15:31,QUANTUM,-,2033-04-08,-888962.01,RUB,-10297.254836,3.0,...,Y,2024-07-31,200686,V100,1000147688,None,2024-09-16,2024-09-16 23:14:48,209390,2024-09-16


In [9]:
assert len(data_Debt_export[data_Debt_export.entity.isna()]) == 0 # data quality
data_Debt_work = data_Debt_export.copy() # copy data
# merge
data_Debt_work['entity_code'] = merge_Mapping(data_Debt_work, col='entity')
data_Debt_work.loc[data_Debt_work.entity_code=='External', 'entity_code'] = data_Debt_work.loc[data_Debt_work.entity_code=='External', 'entity']
data_Debt_work['Entity_group'] = merge_SalesUnits(data_Debt_work, col='entity_code', merge_col='ocpSegment')

# Выполнение цикла для рассчета amount, amountUSD и присваивания значения counterparty для каждого DealSet

deals_data = data_Debt_work[['entity_code', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet','reportDate']] # Выбор нужного разреза данных
deals_data_Bonds = deals_data[deals_data.dealSet.str.contains('Bond')] # Фильтрация разреза по значениям в DealSet, которые содержат слово Bond

Bonds_list = deals_data_Bonds.dealSet.unique()

for day in dates:
    day_data = deals_data_Bonds[deals_data_Bonds.reportDate==day]
    for Bond in Bonds_list:
        try:
            Bond_data = day_data[day_data.dealSet==Bond] # Отбор данных одного Bond
            assert len(Bond_data[Bond_data.instrumentOwner == 'EC_ISSUED_BONDS']) != 0

            columns_list = ['amount', 'amountUSD']
            for column in columns_list:
                Traded_data_sum = float(Bond_data.loc[Bond_data.instrumentOwner=='EC_TRADED_BONDS', column].sum())
                Issued_data = float(Bond_data.loc[Bond_data.instrumentOwner=='EC_ISSUED_BONDS', column].item())
                # Внесение нового значения amount в issued bonds
                Bond_data.loc[Bond_data['instrumentOwner'] =="EC_ISSUED_BONDS", column] = Traded_data_sum + Issued_data 
            # Внесение нового значения counterparty в traded bonds
            Issied_data_entity = Bond_data.loc[Bond_data.instrumentOwner=='EC_ISSUED_BONDS', 'entity_code'].squeeze()
            Bond_data.loc[Bond_data['instrumentOwner'] =="EC_TRADED_BONDS", 'counterparty'] = Issied_data_entity
            day_data.loc[day_data['dealSet'] == Bond] = Bond_data
        except:
            continue
    deals_data_Bonds[deals_data_Bonds.reportDate==day] = day_data

deals_data.loc[deals_data.dealSet.str.contains('Bond')] = deals_data_Bonds
data_Debt_work[['entity_code', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet','reportDate']] = deals_data
# Запись выполненного цикла в в основные данные

data_Debt_work = Period(data_Debt_work, day_for_count='reportDate', col_with_date='actionDate', day_is_col=True).reset_index(drop=True)

data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty_name'] = data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty'].str[:-5] 
# Обрезка " (EC)" в каждой строке

assert len(data_Debt_export[data_Debt_export.counterparty.isna()])==0
data_Debt_work['cty_Code'] = merge_Mapping(data_Debt_work, col='counterparty_name')
data_Debt_work['Counterparty_Group'] = merge_SalesUnits(data_Debt_work, col='cty_Code', merge_col='ocpSegment')
# Merge counterparty с Mapping и SalesUnits

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['SGC' 'SGC' 'SGC' ... 'SGC' 'SGC' 'SGC']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', f'{id_colmn}_merge']
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_14100\2701374154.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [18]:
# instrumentOwner_list = ['EC_ISSUED_BONDS', 'EC_TRADED_BONDS']
# deals_data_Bonds[(deals_data_Bonds.dealSet==Bonds_list[0])&(deals_data_Bonds.reportDate==dates[0])&(deals_data_Bonds.instrumentOwner==instrumentOwner_list[1])]
# data_Debt_work.loc[(data_Debt_work.dealSet==Bonds_list[0])&(data_Debt_work.reportDate==dates[0])&(data_Debt_work.instrumentOwner==instrumentOwner_list[1]), ['entity_code','Entity_group','counterparty','amount','amountUSD','instrumentOwner','dealSet','reportDate']]
# data_Debt_export.loc[(data_Debt_export.instrumentOwner=='EC_TRADED_BONDS')&(data_Debt_export.dealSet=='SUEK EuBond 2021 $500m B'), ['entity','counterparty','instrumentOwner','dealSet','reportDate']]

Деривативы:

In [67]:
query = f"""select * 
from "RISKACCESS"."quantumForwards" 
where "positionDate" in ({dates_SQL})"""
data_Forwards_export = export_from_RISKCUSTOM(query)

query = f"""select * 
from "RISKACCESS"."quantumSwaps" 
where "positionDate" in ({dates_SQL})"""
data_Swaps_export = export_from_RISKCUSTOM(query)

data_Deriv_qu = pd.concat([data_Forwards_export,data_Swaps_export])
data_Deriv_qu.head(2)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_16520\367103458.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_Deriv_qu = pd.concat([data_Forwards_export,data_Swaps_export])
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_16520\367103458.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or

,##batch_id,##deleted,##inserted,##origin_name,adjustedProfitLoss,baseCurrency,baseType,checkSettlementDateAlt,contractRate,counterparty,...,recMargin,recPrincipalFreq,recRateBasisCode,recRateBasisFreq,recRateBasisName,recRateFormula,recRatesetFreq,shortInterestRate,shortNPVOfAdjustedProfitLoss,startdate
0,39681195@WHPROD,N,2024-08-31 01:00:15,QUANTUM,-1672.27,USD,CONTRACT,Y,1.367691,Western Alliance US (EC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,39681195@WHPROD,N,2024-08-31 01:00:15,QUANTUM,-435.73,USD,CONTRACT,Y,1.375000,Western Alliance US (EC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [126]:
# проверка вхождения в debt
Deriv_dealNo = data_Deriv_qu.dealNo.unique().tolist()
Debt_dealNo = data_Debt_export.dealNo.unique().tolist()
Deriv_dealNo_new = [x for x in Deriv_dealNo if x not in Debt_dealNo]
data_Deriv_qu_work = data_Deriv_qu[data_Deriv_qu.dealNo.isin(Deriv_dealNo_new)]
# merge
data_Deriv_qu_work['Entity_group'] = merge_SalesUnits(data_Deriv_qu_work, col='entity', merge_col='ocpSegment')
# melt data
melt_data = pd.melt(data_Deriv_qu_work, id_vars=['positionDate','holdingEntity','Entity_group','entity','counterparty','dealNo','payFaceValue','recFaceValue','maturityDate'], value_vars=['payFXCurrency','recFXCurrency'], value_name='Currency').sort_values('dealNo').rename(columns={'recFaceValue':'amount'})
melt_data.loc[melt_data.variable=='payFXCurrency', 'amount'] = melt_data.loc[melt_data.variable=='payFXCurrency', 'payFaceValue'] * -1
melt_data = melt_data.drop(columns=['payFaceValue','variable']).reset_index(drop=True)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'SAMSALES'
 'SAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES'
 'NAMSALES' 'NAMSALES']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', f'{id_colmn}_merge']
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykern

Долг OCP

In [21]:
query = f"""
select * 
from RISKACCESS."unifiedOcp"
where "sourceTable" = 'quantumDebt'
and "reportDate" in ({dates_SQL})
--fetch first 10 rows only
"""
data_OCP_export = export_from_RISKCUSTOM(query)
data_OCP_export.head(2)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


,##batch_id,##deleted,##inserted,##origin_name,reportDate,rowType,sourceSystem,sourceTable,recordId,balanceItem,...,timeBucket,notionalCurrency,volumeNotionalCurrency,effectiveCurrency,effectiveCurrencyRate,volumeEffectiveCurrency,usdRate,volumeUsd,salesUnitsId,accountSubType
0,40171488@WHPROD,N,2024-09-15 23:15:33,WHFLEX,2024-07-31,M,QUANTUM,quantumDebt,1456,On-balance,...,3M-6M,RUB,3.000000e+09,RUB,1,3.000000e+09,0.011634,3.490349e+07,1L00,None
1,40171488@WHPROD,N,2024-09-15 23:15:33,WHFLEX,2024-07-31,M,QUANTUM,quantumDebt,1456,On-balance,...,3M-6M,USD,-4.787743e+07,USD,1,-4.787743e+07,1.000000,-4.787743e+07,1L00,None


In [64]:
### Compairing Debt
# rename
rename_dict = {'ocpSegment':'Entity_group','effectiveCurrency':'eventCurrency','salesUnitsId':'entity_code','volumeEffectiveCurrency':'amount_OCP','volumeUsd':'amountUSD_OCP','counterpartySegment':'Counterparty_Group','counterpartyName':'counterparty','introgroupCounterpartyCode':'cty_Code'}
data_OCP_work = data_OCP_export.copy()
data_OCP_work = data_OCP_work.rename(columns=rename_dict)
data_OCP_work.cty_Code = data_OCP_work.cty_Code.fillna('External')
data_Debt_work.holding = data_Debt_work.holding.replace({'EuroChem':'EUROCHEM'})
# first compairing
Debt_index_list = ['reportDate', 'holding', 'Entity_group', 'entity_code', 'eventCurrency']
Debt_first = data_Debt_work.groupby(as_index=False,dropna=False, by=Debt_index_list)[['amount', 'amountUSD']].agg(sum)
Debt_first.holding = Debt_first.holding.replace({'EuroChem':'EUROCHEM'})
OCP_first = data_OCP_work.groupby(as_index=False, dropna=False, by=Debt_index_list)[['amount_OCP','amountUSD_OCP']].agg(sum)
# merge
First_merge = Debt_first.merge(OCP_first, how='outer', on=Debt_index_list)
First_merge_not_equal = First_merge[First_merge.amount!=First_merge.amount_OCP]

# second compairing
First_merge_not_equal = concat_columns(First_merge_not_equal, columns=Debt_index_list)
concat_list = First_merge_not_equal.concat_columns.tolist()
data_Debt_work = concat_columns(data_Debt_work, columns=Debt_index_list)
data_OCP_work = concat_columns(data_OCP_work, columns=Debt_index_list)

Debt_index_list = ['reportDate', 'holding', 'Entity_group', 'entity_code', 'eventCurrency', 'Counterparty_Group','counterparty','cty_Code']
Debt_second = data_Debt_work[data_Debt_work.concat_columns.isin(concat_list)].groupby(as_index=False,dropna=False, by=Debt_index_list)[['amount', 'amountUSD']].agg(sum)
OCP_second = data_OCP_work[data_OCP_work.concat_columns.isin(concat_list)].groupby(as_index=False, dropna=False, by=Debt_index_list)[['amount_OCP','amountUSD_OCP']].agg(sum)
# merge
Second_merge = Debt_second.merge(OCP_second, how='outer', on=Debt_index_list)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_16520\3908062160.py:10: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  Debt_first = data_Debt_work.groupby(as_index=False,dropna=False, by=Debt_index_list)[['amount', 'amountUSD']].agg(sum)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_16520\3908062160.py:12: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  OCP_first = data_OCP_work.groupby(as_index=False, dropna=False, by=Debt_index_list)[['amount_OCP','amountUSD_OCP']].agg(sum)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:98: SettingWithCopyWarning: 
A value is trying to be set o

Деривативы OCP

In [68]:
query = f"""
select * 
from RISKACCESS."unifiedOcp"
where "sourceTable" in ('quantumForwards', 'quantumSwaps')
and "reportDate" in ({dates_SQL})
--fetch first 10 rows only
"""
# quantumSwaps
data_Deriv_OCP = export_from_RISKCUSTOM(query)
data_Deriv_OCP.head(2)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


,##batch_id,##deleted,##inserted,##origin_name,reportDate,rowType,sourceSystem,sourceTable,recordId,balanceItem,...,timeBucket,notionalCurrency,volumeNotionalCurrency,effectiveCurrency,effectiveCurrencyRate,volumeEffectiveCurrency,usdRate,volumeUsd,salesUnitsId,accountSubType
0,40045557@WHPROD,N,2024-09-11 15:49:44,WHFLEX,2024-07-31,D,QUANTUM,quantumForwards,8940,Off-balance,...,1M-3M,CAD,300000.00,CAD,1,300000.00,0.724218,217265.353418,USBFC,None
1,40045557@WHPROD,N,2024-09-11 15:49:44,WHFLEX,2024-07-31,D,QUANTUM,quantumForwards,8940,Off-balance,...,1M-3M,USD,-219764.12,USD,1,-219764.12,1.000000,-219764.120000,USBFC,None


In [133]:
# проверка вхождения в debt
Deriv_dealNo = data_Deriv_OCP.recordId.unique().tolist()
Debt_dealNo = data_OCP_export.recordId.unique().tolist()
Deriv_dealNo_new = [x for x in Deriv_dealNo if x not in Debt_dealNo]
data_Deriv_OCP_work = data_Deriv_OCP[data_Deriv_OCP.recordId.isin(Deriv_dealNo_new)]
data_Deriv_OCP_work.shape, data_Deriv_OCP.shape

((28, 40), (64, 40))

In [147]:
data_Deriv_OCP_work = data_Deriv_OCP_work[['reportDate','holding','ocpSegment','buCode','salesUnitsId','counterpartyName','recordId','effectiveCurrency','volumeEffectiveCurrency','maturityDateTo']]
# merge
melt_data_cols = ['positionDate','holdingEntity','Entity_group','entity','counterparty','dealNo','Currency'] # index col
melt_data.holdingEntity = melt_data.holdingEntity.replace({'EuroChem':'EUROCHEM'}) # replace
# rename
rename_dict = {'reportDate':'positionDate','holding':'holdingEntity','ocpSegment':'Entity_group','buCode':'entity','counterpartyName':'counterparty','recordId':'dealNo','effectiveCurrency':'Currency','volumeEffectiveCurrency':'amount_OCP','maturityDateTo':'maturityDateTo_OCP'} 
data_Deriv_OCP_work = data_Deriv_OCP_work.rename(columns=rename_dict)
# dealNo int
melt_data['dealNo'] = melt_data['dealNo'].astype(int)
data_Deriv_OCP_work['dealNo'] = data_Deriv_OCP_work['dealNo'].astype(int)

Deriv_merge = melt_data.merge(data_Deriv_OCP_work, how='outer', on=melt_data_cols)

,positionDate,holdingEntity,Entity_group,entity,counterparty,dealNo,amount,maturityDate,Currency,salesUnitsId,amount_OCP,maturityDateTo_OCP


In [148]:
### to excel
if print_Debt==True:
    Output_file = f'{str(date.today())}_quDebt_vs_unOCP.xlsx'
    with pd.ExcelWriter(Output_file, engine='openpyxl') as writer:
        data_Debt_work.to_excel(writer, index=False, sheet_name='quDebt')
        data_OCP_export.to_excel(writer, index=False, sheet_name='unOCP')
        First_merge.to_excel(writer, index=False, sheet_name='compare_debt_BU')
        Second_merge.to_excel(writer, index=False, sheet_name='compare_debt_cpty')
        Deriv_merge.to_excel(writer, index=False, sheet_name='compare_deriv')